Get general data for the top cities (recent news and current weather)
New York, Chicago, Baltimore, and Atlanta will be selected (but can easily upscale and produce more topics / cities)

In [1]:
from weather import get_city_weather
from news import get_city_news

topics = {
    "New York": (40.7128, -74.0060),
    "Chicago": (41.8781, -87.6298),
    "Baltimore": (39.2904, -76.6122),
    "Atlanta": (33.7490, -84.3880)
}

print("Get city data:")
data = {}
for city, coords in topics.items():
    print(f"Getting data for {city}: Latitude {coords[0]}, Longitude {coords[1]}")
    city_news = get_city_news(city)
    city_data = city_news['articles']
    city_data.append(get_city_weather(coords[0],coords[1]))
    data[city] = city_data


Get city data:
Getting data for New York: Latitude 40.7128, Longitude -74.006
Getting data for Chicago: Latitude 41.8781, Longitude -87.6298
Getting data for Baltimore: Latitude 39.2904, Longitude -76.6122
Getting data for Atlanta: Latitude 33.749, Longitude -84.388


Initialize AMQ producer for the city topics

In [2]:
import time
import stomp
import json

class CityDataProducer(stomp.ConnectionListener):
    def __init__(self, conn):
        self.conn = conn

    def on_error(self, headers, message):
        print('Received an error "%s"' % message)

    def on_disconnected(self):
        print('Lost connection to the broker')

# Create a connection to the ActiveMQ broker
conn = stomp.Connection([('amq', 61613)])
conn.set_listener('', CityDataProducer(conn))
conn.connect()

Send city data to the broker

In [6]:

# Send messages
for city, messages in data.items():
    topic = '/topic/' + city
    for m in messages:
        print("sending on " + topic)
        if city == "New York" or city == "Chicago" :
            if messages[-1] == m:
                print("sending weather data")
                conn.send(body=json.dumps(m),destination=topic,headers={'data': 'weather'})
            else:
                print("sending news data")
                conn.send(body=json.dumps(m), destination=topic,headers={'data': 'news'})
        else:
            if messages[-1] == m:
                print("sending weather data")
                conn.send(body=json.dumps(m),destination=topic,persistent='true',headers={'data': 'weather'})
            else:
                print("sending news data")
                conn.send(body=json.dumps(m), destination=topic,persistent='true',headers={'data': 'news'})
        
# Disconnect from the broker
#conn.disconnect()

sending on /topic/New York
sending news data
sending on /topic/New York
sending news data
sending on /topic/New York
sending news data
sending on /topic/New York
sending news data
sending on /topic/New York
sending news data
sending on /topic/New York
sending news data
sending on /topic/New York
sending news data
sending on /topic/New York
sending news data
sending on /topic/New York
sending news data
sending on /topic/New York
sending news data
sending on /topic/New York
sending news data
sending on /topic/New York
sending news data
sending on /topic/New York
sending news data
sending on /topic/New York
sending news data
sending on /topic/New York
sending news data
sending on /topic/New York
sending news data
sending on /topic/New York
sending news data
sending on /topic/New York
sending news data
sending on /topic/New York
sending news data
sending on /topic/New York
sending news data
sending on /topic/New York
sending news data
sending on /topic/New York
sending news data
sending on